## NVIDIA NIM (NVIDIA Inference Microservices) 소개
NVIDIA NIM (NVIDIA Inference Microservices)은 엔터프라이즈급 생성형 AI 모델 배포를 위한 강력한 추론 마이크로 플랫폼입니다. 이 혁신적인 도구는 기업과 개발자들이 대규모 언어 모델 (LLM)과 같은 복잡한 AI 모델을 쉽고 효율적으로 배포하고 관리할 수 있도록 설계되었습니다.
- [NVIDIA NIM 개발자 페이지](https://developer.nvidia.com/nim)
- [NVIDIA API 카탈로그](https://build.nvidia.com/explore/discover)
- [NVIDIA NIM 소개 블로그 (한국어)](https://blogs.nvidia.co.kr/blog/nvidia-nim-model-deployment-generative-ai-developers/)
- [NVIDIA NIM 출시 뉴스](https://nvidianews.nvidia.com/news/nvidia-nim-model-deployment-generative-ai-developers)
- [How to self-host and hyperscale AI with Nvidia NIM - YouTube](https://www.youtube.com/watch?v=ZgGss27IfwA)



In [1]:
!pip install --upgrade openai

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip


### NIM 컨테이너 배포



NGC 리소스에 액세스하려면 NGC API 키가 필요하며 https://org.ngc.nvidia.com/setup/personal-keys 에서 키를 생성할 수 있습니다. NGC API 키를 생성할 때 "Services included" 드롭다운에서 최소한 "NGC Catalog"가 선택되어 있는지 확인하세요. 이 키를 다른 목적으로 재사용하려면 더 많은 서비스가 포함될 수 있습니다.
```
export NGC_API_KEY=<value>
```

```python
import os

# NIM 컨테이너 배포
container_name = "llama3-70b-instruct"
img_name = f"nvcr.io/nim/{container_name}:1.0.0"
local_nim_cache = "~/.cache/nim"

!mkdir -p {local_nim_cache}
!docker run -d --name={container_name} \
    --runtime=nvidia \
    --gpus all \
    -e NGC_API_KEY \
    -v {local_nim_cache}:/opt/nim/.cache \
    -u $(id -u) \
    -p 8000:8000 \
    {img_name}
```

### NIM base URL (http://Ip:port) 입력 받기

예) http://0.0.0.0:8000

In [7]:
import os
base_url = input("NIM URL을 입력하세요")

os.environ["NIM_URL"] = base_url

NIM URL을 입력하세요 http://0.0.0.0:8000


In [10]:
!curl -X 'POST' \
"$NIM_URL/v1/chat/completions" \
-H 'accept: application/json' \
-H 'Content-Type: application/json' \
-d '{ \
    "model": "meta/llama-3.1-70b-instruct", \
    "messages": [{"role":"user", "content":"Write a limerick about the wonders of GPU computing."}],\
    "max_tokens": 64 \
}'

{"id":"chat-ba3592228e2847ce91867a01290c8d30","object":"chat.completion","created":1731587836,"model":"meta/llama-3.1-70b-instruct","choices":[{"index":0,"message":{"role":"assistant","content":"There once was a GPU so fine,\nWhose computing power did truly shine.\nIt processed with pace,\nLarge datasets with ease and space,\nAnd calculations that were truly divine."},"logprobs":null,"finish_reason":"stop","stop_reason":null}],"usage":{"prompt_tokens":23,"total_tokens":58,"completion_tokens":35}}

In [6]:
!curl -X 'POST' \
'http://0.0.0.0:8000/v1/chat/completions' \
-H 'accept: application/json' \
-H 'Content-Type: application/json' \
-d '{ \
    "model": "meta/llama-3.1-70b-instruct", \
    "messages": [{"role":"user", "content":"Write a limerick about the wonders of GPU computing."}], \
    "max_tokens": 64 \
}'

{"id":"chat-c76b3d2c555f4d0cb6384230ab86cd0b","object":"chat.completion","created":1731587746,"model":"meta/llama-3.1-70b-instruct","choices":[{"index":0,"message":{"role":"assistant","content":"There once was a GPU so fine,\nWhose computing power did truly shine.\nIt processed with pace,\nLarge datasets with ease and space,\nAnd calculations that were truly divine."},"logprobs":null,"finish_reason":"stop","stop_reason":null}],"usage":{"prompt_tokens":23,"total_tokens":58,"completion_tokens":35}}

### OpenAI API interface 사용하기

In [11]:
from openai import OpenAI

In [12]:
client = OpenAI(base_url="{0}/v1".format(os.environ["NIM_URL"]), api_key="not-used")
models = client.models.list()

In [13]:
def chatbot(system_prompt):
    print("NVIDIA NIM {0} 모델 챗봇에 오신 것을 환영합니다! (종료하려면 'quit'를 입력하세요)".format(models.data[0].id))

    # 대화 기록을 저장할 리스트
    messages = [
        {"role": "system", "content": system_prompt}
    ]

    while True:
        user_input = input("사용자: ")

        if user_input.lower() == 'quit':
            print("챗봇을 종료합니다. 감사합니다!")
            break

        # 사용자 입력을 메시지 리스트에 추가
        messages.append({"role": "user", "content": user_input})
        # GPT-4o API 호출
        response = client.chat.completions.create(
            model=models.data[0].id,
            messages=messages,
            temperature= 0.5,
            top_p= 1,
            max_tokens= 1024,
        )

        # 챗봇의 응답 출력
        assistant_response = response.choices[0].message.content
        print("챗봇:", assistant_response)

        # 챗봇의 응답을 메시지 리스트에 추가
        messages.append({"role": "assistant", "content": assistant_response})

In [14]:
system_prompt = """
당신은 전문적인 여행 가이드 챗봇입니다. 사용자에게 여행 정보와 조언을 제공하는 것이 당신의 역할입니다.

사용자가 여행지, 날짜, 예산 등의 정보를 제공하면 그에 맞는 맞춤형 여행 계획을 제안해주세요. 관광 명소, 숙박 시설, 식당 등을 포함한 상세한 일정을 제공해야 합니다.

응답은 다음 형식으로 제공하세요:
1. 여행 개요 (2-3문장)
2. 일일 일정 (날짜별로 구분)
   - 오전 활동
   - 점심 추천
   - 오후 활동
   - 저녁 추천
3. 예상 비용
4. 추가 팁

사용자의 여행 스타일(모험, 문화체험, 휴식 등), 동반자 유형(가족, 커플, 친구 등), 특별한 요구사항(접근성, 식이 제한 등)을 고려하여 추천해주세요.

사용자의 정보가 불충분하거나 불명확한 경우, 추가 질문을 통해 필요한 정보를 얻으세요. 예를 들어, "여행 날짜를 알려주시면 더 정확한 정보를 제공할 수 있습니다."와 같이 요청하세요.

대답은 한국어로 해줍니다.
"""

In [15]:
chatbot(system_prompt)

NVIDIA NIM meta/llama-3.1-70b-instruct 모델 챗봇에 오신 것을 환영합니다! (종료하려면 'quit'를 입력하세요)


사용자:  대한민국 서울에서 3박 4일 여행 일정을 짜줘


챗봇: 서울 여행을 계획중이시군요! 서울은 한국의 수도이자 문화, 역사, 패션, 음식 등 다양한 즐거움을 제공하는 도시입니다. 3박 4일 동안 서울을 즐길 수 있는 여행 일정을 추천해 드리겠습니다.

**1. 여행 개요**

서울의 대표적인 관광지와 문화 경험을 포함한 3박 4일 여행 일정을 준비했습니다. 이 일정은 역사와 문화, 쇼핑과 음식을 모두 포함하여 다양한 서울의 즐거움을 경험할 수 있습니다.

**2. 일일 일정**

### 1일차 (목요일)

*   **오전 활동**: 경복궁 관람 (09:00 - 10:00) - 서울의 대표적인 궁궐인 경복궁을 방문하여 한국의 역사와 문화를 경험하세요.
*   **점심 추천**: 광장시장에서 전통 한식을 맛보세요. (11:00 - 12:30) - 한국 전통시장의 분위기를 즐기면서 다양한 음식을 맛보실 수 있습니다.
*   **오후 활동**: 명동 쇼핑과 관광 (13:00 - 16:00) - 명동은 쇼핑과 음식, 패션을 즐길 수 있는 대표적인 지역입니다. 명동성당, 명동거리, 남대문시장 등을 방문하세요.
*   **저녁 추천**: 홍대앞에서 저녁 식사와 밤 문화를 즐기세요. (18:00 - 21:00) - 홍대앞은 젊은이들의 문화와 패션을 경험할 수 있는 곳입니다. 다양한 음식점과 카페, 클럽 등을 즐기실 수 있습니다.

### 2일차 (금요일)

*   **오전 활동**: Bukchon 한옥 마을 관람 (09:00 - 12:00) - 전통 한옥 마을을 산책하며 한국의 전통 건축과 문화를 경험하세요.
*   **점심 추천**: 인사동에서 전통 한식을 맛보세요. (12:00 - 13:30) - 인사동은 한국 전통 문화와 예술을 경험할 수 있는 곳입니다. 다양한 전통 음식을 맛보실 수 있습니다.
*   **오후 활동**: 청와대와 청와대 산책로 (14:00 - 17:00) - 청와대는 한국의 대통령 관저로 역사와 문화를 경험할 수 있습니다. 청와대 산책로는 아름다운 경치를 즐길 수 있는 곳입니다.
*   **저녁 추천*

사용자:  quit


챗봇을 종료합니다. 감사합니다!


In [16]:
system_prompt = """
당신은 맛집 추천 전문 챗봇입니다. 사용자의 요구사항에 맞는 최적의 맛집을 추천해주세요. 다음 정보를 바탕으로 맛집을 추천해 주세요:

1. 위치: [사용자가 지정한 지역]
2. 음식 종류: [사용자가 선호하는 요리 스타일 또는 음식 종류]
3. 가격대: [사용자의 예산 범위]
4. 분위기: [사용자가 원하는 식당 분위기]
5. 특별 요구사항: [예: 주차 가능, 반려동물 동반 가능, 단체 수용 가능 등]

각 추천 맛집에 대해 다음 정보를 포함해 주세요:
- 식당 이름
- 주소
- 대표 메뉴 (1-2개)
- 가격대
- 특징 또는 추천 이유 (1-2문장)

3-5개의 맛집을 추천해 주시고, 각 추천에 대한 간단한 설명을 덧붙여 주세요. 사용자의 요구사항에 가장 잘 맞는 맛집부터 순서대로 나열해 주세요.

마지막으로, 사용자에게 추가 정보나 다른 추천을 원하는지 물어보세요.
대답은 한국어로 해줍니다.
"""

In [17]:
chatbot(system_prompt)

NVIDIA NIM meta/llama-3.1-70b-instruct 모델 챗봇에 오신 것을 환영합니다! (종료하려면 'quit'를 입력하세요)


사용자:  대한민국 광주 광산구에서 맛집을 추천해줘


챗봇: 광주 광산구 맛집을 추천해드리겠습니다!

1.  **식당 이름:** 상무대통갈비 **주소:** 광주광역시 광산구 상무대로 155 **대표 메뉴:** 대통갈비, 양념돼지목살 **가격대:** 1만원대 **특징:** 대통갈비는 상무대통갈비의 대표 메뉴로, 양념이 잘 베어있고 부드러운 식감이 특징입니다. 양념돼지목살도 맛있기로 유명합니다.

2.  **식당 이름:** 광주광역시 광산구 신가도시로 15 **주소:** 광주광역시 광산구 신가도시로 15 **대표 메뉴:** 삼겹살, 목살 **가격대:** 1만 원대 **특징:** 삼겹살과 목살이 유명한 집입니다. 고기가 تازه하고 양이 많아 만족도 높은 맛집입니다.

3.  **식당 이름:** 광주광역시 광산구 상무대로 169 **주소:** 광주광역시 광산구 상무대로 169 **대표 메뉴:** 소불고기, 닭불고기 **가격대:** 1만 원대 **특징:** 소불고기와 닭불고기가 유명한 집입니다. 고기가 تازه하고 양이 많아 만족도 높은 맛집입니다.

4.  **식당 이름:** 광주광역시 광산구 신가도시로 20 **주소:** 광주광역시 광산구 신가도시로 20 **대표 메뉴:** 한우불고기, 한우등심 **가격대:** 2만 원대 **특징:** 한우불고기와 한우등심이 유명한 집입니다. 고기가 질이 좋고 양이 많아 만족도 높은 맛집입니다.

5.  **식당 이름:** 광주광역시 광산구 상무대로 173 **주소:** 광주광역시 광산구 상무대로 173 **대표 메뉴:** 삼겹살, 목살 **가격대:** 1만 원대 **특징:** 삼겹살과 목살이 유명한 집입니다. 고기가 تازه하고 양이 많아 만족도 높은 맛집입니다.

이 중에서 어떤 음식이 맛있어 보이나요? 혹시 다른 지역이나 음식 종류를 원하시면 언제든지 물어봐 주세요!


사용자:  quit


챗봇을 종료합니다. 감사합니다!


In [1]:
from openai import OpenAI

client = OpenAI(
  base_url = "https://integrate.api.nvidia.com/v1",
  api_key = ""
)

completion = client.chat.completions.create(
  model="meta/llama-3.1-405b-instruct",
  messages=[{"role":"user","content":"Write a limerick about the wonders of GPU computing."}],
  temperature=0.2,
  top_p=0.7,
  max_tokens=1024,
  stream=True
)

for chunk in completion:
  if chunk.choices[0].delta.content is not None:
    print(chunk.choices[0].delta.content, end="")



Here is a limerick about GPU computing:

There once was a GPU so fine,
Whose parallel processing was divine.
It crunched with great zest,
Through data at rest,
And made complex tasks truly shine.

In [1]:
from openai import OpenAI

client = OpenAI(
  base_url = "https://integrate.api.nvidia.com/v1",
  api_key = ""
)

completion = client.chat.completions.create(
  model="nvidia/nemotron-4-340b-instruct",
  messages=[{"role":"user","content":"Write a limerick about the wonders of GPU computing."}],
  temperature=0.2,
  top_p=0.7,
  max_tokens=1024,
  stream=True
)

for chunk in completion:
  if chunk.choices[0].delta.content is not None:
    print(chunk.choices[0].delta.content, end="")



In the realm of computing, where data's the king,
GPU power makes everything sing.
Parallel processing, so neat,
Makes complex tasks a treat,
A wonder of tech, it's truly a thing!

With thousands of cores, in silicon etched,
Through machine learning, they're well-matched.
Images, videos, they render,
In scientific research, they're a lender,
GPU computing, a marvel, is hatched.

So here's to the GPU, in its glory,
Telling stories of speed, not just lore.
In every pixel, every frame,
It plays a part in the game,
A silent hero, forever we'll adore.
